## Подключение к Google Cloud

In [ ]:
conda install pandas-gbq -c conda-forge

In [2]:
pip install pandas-gbq



You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd

In [3]:
from google.oauth2 import service_account

In [4]:
credentials = service_account.Credentials.from_service_account_file(r'C:\Users\chafi\Downloads\My Project 38885-205f6c5e091a.json')

In [5]:
query = '''

WITH
  session_data AS (
  SELECT
    date,
    fullVisitorId,
    visitNumber,
    IF(trafficSource.isTrueDirect, "(direct)", trafficSource.source) AS Source,
    IF(trafficSource.isTrueDirect, "(none)", trafficSource.medium) AS Medium,
    IF(trafficSource.isTrueDirect, "(not set)", trafficSource.campaign) AS Campaign,
    IF(trafficSource.isTrueDirect, "Direct", channelGrouping) AS Channel_Group,
    totals.transactions AS Transactions,
    totals.totalTransactionRevenue AS Revenue,
    
    
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS hit
  WHERE
    _TABLE_SUFFIX >= '20170501'
    AND _TABLE_SUFFIX <= '20170530'
  GROUP BY
    date,
    fullVisitorId,
    visitNumber,
    Source, 
    Medium,
    Campaign,
    Channel_Group,
    Transactions,
    Revenue) (
  WITH
    path_data AS (
    SELECT
      fullVisitorId AS Google_client_Id,
      STRING_AGG(Channel_Group, ' > ') OVER (PARTITION BY fullVisitorId ORDER BY visitNumber ROWS UNBOUNDED PRECEDING) AS Channel_funnel,
      STRING_AGG(Medium, ' > ') OVER (PARTITION BY FullVisitorId ORDER BY visitNumber ROWS UNBOUNDED PRECEDING) AS Medium_funnel,
      STRING_AGG(source, ' > ') OVER (PARTITION BY FullVisitorId ORDER BY visitNumber ROWS UNBOUNDED PRECEDING) AS Source_funnel,
      STRING_AGG(Campaign, ' > ') OVER (PARTITION BY FullVisitorId ORDER BY visitNumber ROWS UNBOUNDED PRECEDING) AS Campaign_funnel,
      SUM(Transactions) AS Transactions,
      SUM(Revenue/1000000) AS Revenue,
    FROM
      session_data AS data
    GROUP BY
      Google_client_Id,
      Channel_Group,
      visitNumber,
      Medium,
      Source,
      Campaign)
  SELECT
    *
  FROM
    path_data)
'''

In [6]:
project_id = 'quantum-genre-299519'

In [7]:
df = pd.read_gbq(query, project_id=project_id, credentials=credentials)

## Работа с DataFrame

In [8]:
display(df.head(1000))

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue
0,0000197671390269035,Social,referral,m.facebook.com,(not set),NaN,NaN
1,0000458812883559498,Organic Search,organic,google,(not set),NaN,NaN
2,0000572434142265465,Referral,(none),(direct),(not set),NaN,NaN
3,0001436786657417059,Organic Search,organic,google,(not set),NaN,NaN
4,0001611849527956932,Direct,(none),(direct),(not set),NaN,NaN
...,...,...,...,...,...,...,...
995,0160608006828160561,Social,referral,youtube.com,(not set),NaN,NaN
996,0160624362519211771,Organic Search,organic,google,(not set),NaN,NaN
997,0161235414656541525,Organic Search,organic,google,(not set),NaN,NaN
998,0161326209203992257,Organic Search,organic,google,(not set),NaN,NaN


### Добавление нвого столбца New_Transactions

In [9]:
df['New_Transactions'] = df['Transactions'] 

In [10]:
df

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
0,0000197671390269035,Social,referral,m.facebook.com,(not set),NaN,NaN,NaN
1,0000458812883559498,Organic Search,organic,google,(not set),NaN,NaN,NaN
2,0000572434142265465,Referral,(none),(direct),(not set),NaN,NaN,NaN
3,0001436786657417059,Organic Search,organic,google,(not set),NaN,NaN,NaN
4,0001611849527956932,Direct,(none),(direct),(not set),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
62980,9999350616287088509,Organic Search,organic,google,(not set),NaN,NaN,NaN
62981,9999355589118867468,Referral,referral,analytics.google.com,(not set),NaN,NaN,NaN
62982,9999355589118867468,Referral > Organic Search,referral > organic,analytics.google.com > google,(not set) > (not set),NaN,NaN,NaN
62983,9999520059803839410,Organic Search,organic,google,(not set),NaN,NaN,NaN


### Сортировка по возрастанию по столбику New_Transactions

In [11]:
df.sort_values(by='New_Transactions', ascending=False)

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
30849,4921413806940784676,Direct > Direct > Direct > Direct > Organic Se...,(none) > (none) > (none) > (none) > organic > ...,(direct) > (direct) > (direct) > (direct) > go...,(not set) > (not set) > (not set) > (not set) ...,4.0,608.59,4.0
14998,2402527199731150932,Direct,(none),(direct),(not set),4.0,246.18,4.0
31312,4988517937139937145,Direct,(none),(direct),(not set),3.0,63.97,3.0
56783,9026840718082010040,Direct > Direct > Direct > Direct > Direct > D...,(none) > (none) > (none) > (none) > (none) > (...,(direct) > (direct) > (direct) > (direct) > (d...,(not set) > (not set) > (not set) > (not set) ...,3.0,2298.60,3.0
24251,3882155860208938967,Direct > Referral > Direct,(none) > (none) > (none),(direct) > (direct) > (direct),(not set) > (not set) > (not set),2.0,559.51,2.0
...,...,...,...,...,...,...,...,...
62980,9999350616287088509,Organic Search,organic,google,(not set),NaN,NaN,NaN
62981,9999355589118867468,Referral,referral,analytics.google.com,(not set),NaN,NaN,NaN
62982,9999355589118867468,Referral > Organic Search,referral > organic,analytics.google.com > google,(not set) > (not set),NaN,NaN,NaN
62983,9999520059803839410,Organic Search,organic,google,(not set),NaN,NaN,NaN


### Проверка типа данных для столбцов

In [12]:
df.dtypes

Google_client_Id     object
Channel_funnel       object
Medium_funnel        object
Source_funnel        object
Campaign_funnel      object
Transactions        float64
Revenue             float64
New_Transactions    float64
dtype: object

### Изменем тип данных для стобца New_Transactions на str, это нужно для изменения данных

In [13]:
df[['New_Transactions']] = df[['New_Transactions']].astype(str)

In [14]:
df.dtypes

Google_client_Id     object
Channel_funnel       object
Medium_funnel        object
Source_funnel        object
Campaign_funnel      object
Transactions        float64
Revenue             float64
New_Transactions     object
dtype: object

### Следующая функция меняет значение в конкретном столбце

In [15]:
df["New_Transactions"].replace({"4.0": "1.0"}, inplace=True)

In [16]:
df.sort_values(by='New_Transactions', ascending=True)

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
37857,6023796381284015578,Direct,(none),(direct),(not set),1.0,40.59,1.0
56585,8997285863534192253,Referral > Direct,(none) > (none),(direct) > (direct),(not set) > (not set),1.0,117.91,1.0
10888,1749853719127085935,Direct,(none),(direct),(not set),1.0,24.99,1.0
34480,5483156497557634103,Social > Direct,referral > (none),groups.google.com > (direct),(not set) > (not set),1.0,34.59,1.0
17380,2778312825087268268,Direct,(none),(direct),(not set),1.0,69.36,1.0
...,...,...,...,...,...,...,...,...
21172,3385501868972757824,Referral,(none),(direct),(not set),NaN,NaN,nan
21173,3385501868972757824,Referral > Direct,(none) > (none),(direct) > (direct),(not set) > (not set),NaN,NaN,nan
21174,3385501868972757824,Referral > Direct > Direct,(none) > (none) > (none),(direct) > (direct) > (direct),(not set) > (not set) > (not set),NaN,NaN,nan
21146,3380543152349331595,Organic Search > Direct,organic > (none),google > (direct),(not set) > (not set),NaN,NaN,nan


### Следующая функция нужна для проверки по конкретной строке в данном случае по Revenue и значению 608.59

In [17]:
df[df['Revenue'].isin(['608.59'])]

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
30849,4921413806940784676,Direct > Direct > Direct > Direct > Organic Se...,(none) > (none) > (none) > (none) > organic > ...,(direct) > (direct) > (direct) > (direct) > go...,(not set) > (not set) > (not set) > (not set) ...,4.0,608.59,1.0


In [18]:
df["New_Transactions"].replace({"3.0": "1.0"}, inplace=True)

In [19]:
df[df['Revenue'].isin(['2298.6'])]

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
56783,9026840718082010040,Direct > Direct > Direct > Direct > Direct > D...,(none) > (none) > (none) > (none) > (none) > (...,(direct) > (direct) > (direct) > (direct) > (d...,(not set) > (not set) > (not set) > (not set) ...,3.0,2298.6,1.0


In [20]:
df["New_Transactions"].replace({"2.0": "1.0"}, inplace=True)

In [21]:
df[df['Revenue'].isin(['559.51'])]

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
24251,3882155860208938967,Direct > Referral > Direct,(none) > (none) > (none),(direct) > (direct) > (direct),(not set) > (not set) > (not set),2.0,559.51,1.0


In [22]:
df["New_Transactions"].replace({"nan": "2.0"}, inplace=True)

In [23]:
df[df['Google_client_Id'].isin(['6711117305274413893'])]

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
42187,6711117305274413893,Organic Search,organic,google,(not set),NaN,NaN,2.0


In [24]:
df["New_Transactions"].replace({"1.0": ""}, inplace=True)

In [25]:
df[df['Revenue'].isin(['559.51'])]

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
24251,3882155860208938967,Direct > Referral > Direct,(none) > (none) > (none),(direct) > (direct) > (direct),(not set) > (not set) > (not set),2.0,559.51,


In [26]:
df[df['Revenue'].isin(['2298.6'])]

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
56783,9026840718082010040,Direct > Direct > Direct > Direct > Direct > D...,(none) > (none) > (none) > (none) > (none) > (...,(direct) > (direct) > (direct) > (direct) > (d...,(not set) > (not set) > (not set) > (not set) ...,3.0,2298.6,


In [27]:
df["New_Transactions"].replace({"2.0": "1.0"}, inplace=True)

In [28]:
df[df['Google_client_Id'].isin(['6711117305274413893'])]

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
42187,6711117305274413893,Organic Search,organic,google,(not set),NaN,NaN,1.0


### Следующая функция меняет тип данных для столбца New_Transactions на float

In [29]:
df['New_Transactions'] = pd.to_numeric(df['New_Transactions'], errors='coerce')
df.dtypes

Google_client_Id     object
Channel_funnel       object
Medium_funnel        object
Source_funnel        object
Campaign_funnel      object
Transactions        float64
Revenue             float64
New_Transactions    float64
dtype: object

In [30]:
df[df['Google_client_Id'].isin(['6711117305274413893'])]

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
42187,6711117305274413893,Organic Search,organic,google,(not set),NaN,NaN,1.0


In [31]:
#Начало
df

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions
0,0000197671390269035,Social,referral,m.facebook.com,(not set),NaN,NaN,1.0
1,0000458812883559498,Organic Search,organic,google,(not set),NaN,NaN,1.0
2,0000572434142265465,Referral,(none),(direct),(not set),NaN,NaN,1.0
3,0001436786657417059,Organic Search,organic,google,(not set),NaN,NaN,1.0
4,0001611849527956932,Direct,(none),(direct),(not set),NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
62980,9999350616287088509,Organic Search,organic,google,(not set),NaN,NaN,1.0
62981,9999355589118867468,Referral,referral,analytics.google.com,(not set),NaN,NaN,1.0
62982,9999355589118867468,Referral > Organic Search,referral > organic,analytics.google.com > google,(not set) > (not set),NaN,NaN,1.0
62983,9999520059803839410,Organic Search,organic,google,(not set),NaN,NaN,1.0


In [32]:
df_Channel_funnel = df.Channel_funnel.str.split(" > ", expand=True)

In [33]:
df_Channel_funnel

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,Social,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Organic Search,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Referral,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Organic Search,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Direct,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,Organic Search,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62981,Referral,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62982,Referral,Organic Search,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62983,Organic Search,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [34]:
df_Channel_funnel.columns = df_Channel_funnel.columns.map(str)

In [35]:
df_Channel_funnel.dtypes

0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
21    object
22    object
23    object
24    object
25    object
26    object
27    object
28    object
29    object
30    object
31    object
32    object
33    object
34    object
35    object
36    object
37    object
38    object
39    object
40    object
41    object
42    object
dtype: object

In [57]:
#del df_Channel_funnel['0']

In [36]:
df_Channel_funnel = df_Channel_funnel.rename(columns={'0': 'df_Channel_funnel0',
                                                     '1': 'df_Channel_funnel1',
                                                     '2': 'df_Channel_funnel2',
                                                     '3': 'df_Channel_funnel3',
                                                     '4': 'df_Channel_funnel4',
                                                     '5': 'df_Channel_funnel5',
                                                     '6': 'df_Channel_funnel6',
                                                     '7': 'df_Channel_funnel7',
                                                     '8': 'df_Channel_funnel8',
                                                     '9': 'df_Channel_funnel9',
                                                     '10': 'df_Channel_funnel10',
                                                     '11': 'df_Channel_funnel11',
                                                     '12': 'df_Channel_funnel12',
                                                     '13': 'df_Channel_funnel13',
                                                     '14': 'df_Channel_funnel14',
                                                     '15': 'df_Channel_funnel15',
                                                     '16': 'df_Channel_funnel16',
                                                     '17': 'df_Channel_funnel17',
                                                     '18': 'df_Channel_funnel18',
                                                     '19': 'df_Channel_funnel19',
                                                     '20': 'df_Channel_funnel20',
                                                     '21': 'df_Channel_funnel21',
                                                     '22': 'df_Channel_funnel22',
                                                     '23': 'df_Channel_funnel23',
                                                     '24': 'df_Channel_funnel24',
                                                     '25': 'df_Channel_funnel25',
                                                     '26': 'df_Channel_funnel26',
                                                     '27': 'df_Channel_funnel27',
                                                     '28': 'df_Channel_funnel28',
                                                     '29': 'df_Channel_funnel29',
                                                     '30': 'df_Channel_funnel30',
                                                     '31': 'df_Channel_funnel31',
                                                     '32': 'df_Channel_funnel32',
                                                     '33': 'df_Channel_funnel33',
                                                     '34': 'df_Channel_funnel34',
                                                     '35': 'df_Channel_funnel35',
                                                     '36': 'df_Channel_funnel36',
                                                     '37': 'df_Channel_funnel37',
                                                     '38': 'df_Channel_funnel38',
                                                     '39': 'df_Channel_funnel39',
                                                     '40': 'df_Channel_funnel40',
                                                     '41': 'df_Channel_funnel41',
                                                     '42': 'df_Channel_funnel42',})

In [37]:
df_Channel_funnel

,df_Channel_funnel0,df_Channel_funnel1,df_Channel_funnel2,df_Channel_funnel3,df_Channel_funnel4,df_Channel_funnel5,df_Channel_funnel6,df_Channel_funnel7,df_Channel_funnel8,df_Channel_funnel9,...,df_Channel_funnel33,df_Channel_funnel34,df_Channel_funnel35,df_Channel_funnel36,df_Channel_funnel37,df_Channel_funnel38,df_Channel_funnel39,df_Channel_funnel40,df_Channel_funnel41,df_Channel_funnel42
0,Social,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Organic Search,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Referral,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Organic Search,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Direct,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,Organic Search,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62981,Referral,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62982,Referral,Organic Search,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62983,Organic Search,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [62]:
#df_Channel_funnel.to_csv('df_Channel_funnel1.csv')

In [49]:
#df_Channel_funnel.to_csv('df_Channel_funnel.csv')

In [38]:
df_Campaign_funnel = df.Campaign_funnel.str.split(" > ", expand=True)

In [39]:
df_Campaign_funnel

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62981,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62982,(not set),(not set),None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62983,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [40]:
df_Campaign_funnel.columns = df_Campaign_funnel.columns.map(str)

In [43]:
#del df_Campaign_funnel['0']

In [41]:
df_Campaign_funnel = df_Campaign_funnel.rename(columns={'0': 'df_Campaign_funnel0',
                                                     '1': 'df_Campaign_funnel1',
                                                     '2': 'df_Campaign_funnel2',
                                                     '3': 'df_Campaign_funnel3',
                                                     '4': 'df_Campaign_funnel4',
                                                     '5': 'df_Campaign_funnel5',
                                                     '6': 'df_Campaign_funnel6',
                                                     '7': 'df_Campaign_funnel7',
                                                     '8': 'df_Campaign_funnel8',
                                                     '9': 'df_Campaign_funnel9',
                                                     '10': 'df_Campaign_funnel10',
                                                     '11': 'df_Campaign_funnel11',
                                                     '12': 'df_Campaign_funnel12',
                                                     '13': 'df_Campaign_funnel13',
                                                     '14': 'df_Campaign_funnel14',
                                                     '15': 'df_Campaign_funnel15',
                                                     '16': 'df_Campaign_funnel16',
                                                     '17': 'df_Campaign_funnel17',
                                                     '18': 'df_Campaign_funnel18',
                                                     '19': 'df_Campaign_funnel19',
                                                     '20': 'df_Campaign_funnel20',
                                                     '21': 'df_Campaign_funnel21',
                                                     '22': 'df_Campaign_funnel22',
                                                     '23': 'df_Campaign_funnel23',
                                                     '24': 'df_Campaign_funnel24',
                                                     '25': 'df_Campaign_funnel25',
                                                     '26': 'df_Campaign_funnel26',
                                                     '27': 'df_Campaign_funnel27',
                                                     '28': 'df_Campaign_funnel28',
                                                     '29': 'df_Campaign_funnel29',
                                                     '30': 'df_Campaign_funnel30',
                                                     '31': 'df_Campaign_funnel31',
                                                     '32': 'df_Campaign_funnel32',
                                                     '33': 'df_Campaign_funnel33',
                                                     '34': 'df_Campaign_funnel34',
                                                     '35': 'df_Campaign_funnel35',
                                                     '36': 'df_Campaign_funnel36',
                                                     '37': 'df_Campaign_funnel37',
                                                     '38': 'df_Campaign_funnel38',
                                                     '39': 'df_Campaign_funnel39',
                                                     '40': 'df_Campaign_funnel40',
                                                     '41': 'df_Campaign_funnel41',
                                                     '42': 'df_Campaign_funnel42',})

In [42]:
df_Campaign_funnel

,df_Campaign_funnel0,df_Campaign_funnel1,df_Campaign_funnel2,df_Campaign_funnel3,df_Campaign_funnel4,df_Campaign_funnel5,df_Campaign_funnel6,df_Campaign_funnel7,df_Campaign_funnel8,df_Campaign_funnel9,...,df_Campaign_funnel33,df_Campaign_funnel34,df_Campaign_funnel35,df_Campaign_funnel36,df_Campaign_funnel37,df_Campaign_funnel38,df_Campaign_funnel39,df_Campaign_funnel40,df_Campaign_funnel41,df_Campaign_funnel42
0,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62981,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62982,(not set),(not set),None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62983,(not set),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [47]:
#df_Campaign_funnel.to_csv('df_Campaign_funnel')

In [48]:
#res1 = res.merge(df_Campaign_funnel, df_Channel_funnel, how="left", on="location")


res = df.join(df_Campaign_funnel)
res1 = res.join(df_Channel_funnel)
res1

,Google_client_Id,Channel_funnel,Medium_funnel,Source_funnel,Campaign_funnel,Transactions,Revenue,New_Transactions,df_Campaign_funnel0,df_Campaign_funnel1,...,df_Channel_funnel33,df_Channel_funnel34,df_Channel_funnel35,df_Channel_funnel36,df_Channel_funnel37,df_Channel_funnel38,df_Channel_funnel39,df_Channel_funnel40,df_Channel_funnel41,df_Channel_funnel42
0,0000197671390269035,Social,referral,m.facebook.com,(not set),NaN,NaN,1.0,(not set),None,...,None,None,None,None,None,None,None,None,None,None
1,0000458812883559498,Organic Search,organic,google,(not set),NaN,NaN,1.0,(not set),None,...,None,None,None,None,None,None,None,None,None,None
2,0000572434142265465,Referral,(none),(direct),(not set),NaN,NaN,1.0,(not set),None,...,None,None,None,None,None,None,None,None,None,None
3,0001436786657417059,Organic Search,organic,google,(not set),NaN,NaN,1.0,(not set),None,...,None,None,None,None,None,None,None,None,None,None
4,0001611849527956932,Direct,(none),(direct),(not set),NaN,NaN,1.0,(not set),None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,Organic Search,organic,google,(not set),NaN,NaN,1.0,(not set),None,...,None,None,None,None,None,None,None,None,None,None
62981,9999355589118867468,Referral,referral,analytics.google.com,(not set),NaN,NaN,1.0,(not set),None,...,None,None,None,None,None,None,None,None,None,None
62982,9999355589118867468,Referral > Organic Search,referral > organic,analytics.google.com > google,(not set) > (not set),NaN,NaN,1.0,(not set),(not set),...,None,None,None,None,None,None,None,None,None,None
62983,9999520059803839410,Organic Search,organic,google,(not set),NaN,NaN,1.0,(not set),None,...,None,None,None,None,None,None,None,None,None,None


In [49]:
del res1['Campaign_funnel']

In [50]:
del res1['Channel_funnel']

In [51]:
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,df_Campaign_funnel0,df_Campaign_funnel1,df_Campaign_funnel2,df_Campaign_funnel3,...,df_Channel_funnel33,df_Channel_funnel34,df_Channel_funnel35,df_Channel_funnel36,df_Channel_funnel37,df_Channel_funnel38,df_Channel_funnel39,df_Channel_funnel40,df_Channel_funnel41,df_Channel_funnel42
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,0000458812883559498,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,0001436786657417059,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,(not set),(not set),None,None,...,None,None,None,None,None,None,None,None,None,None
62983,9999520059803839410,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [88]:
#res1['Svedeno1'] = res1[['df_Channel_funnel1','df_Campaign_funnel1']].astype(str).apply(' : '.join, axis=1)

In [52]:
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,df_Campaign_funnel0,df_Campaign_funnel1,df_Campaign_funnel2,df_Campaign_funnel3,...,df_Channel_funnel33,df_Channel_funnel34,df_Channel_funnel35,df_Channel_funnel36,df_Channel_funnel37,df_Channel_funnel38,df_Channel_funnel39,df_Channel_funnel40,df_Channel_funnel41,df_Channel_funnel42
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,0000458812883559498,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,0001436786657417059,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,(not set),(not set),None,None,...,None,None,None,None,None,None,None,None,None,None
62983,9999520059803839410,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [91]:
#res1.to_csv('res0.csv')

In [53]:
res1['Svedeno0']=res1['df_Channel_funnel0'].astype(str)+' : '+res1['df_Campaign_funnel0'].astype(str)
res1['Svedeno1']=res1['df_Channel_funnel1'].astype(str)+' : '+res1['df_Campaign_funnel1'].astype(str)
res1['Svedeno2']=res1['df_Channel_funnel2'].astype(str)+' : '+res1['df_Campaign_funnel2'].astype(str)
res1['Svedeno3']=res1['df_Channel_funnel3'].astype(str)+' : '+res1['df_Campaign_funnel3'].astype(str)
res1['Svedeno4']=res1['df_Channel_funnel4'].astype(str)+' : '+res1['df_Campaign_funnel4'].astype(str)
res1['Svedeno5']=res1['df_Channel_funnel5'].astype(str)+' : '+res1['df_Campaign_funnel5'].astype(str)
res1['Svedeno6']=res1['df_Channel_funnel6'].astype(str)+' : '+res1['df_Campaign_funnel6'].astype(str)
res1['Svedeno7']=res1['df_Channel_funnel7'].astype(str)+' : '+res1['df_Campaign_funnel7'].astype(str)
res1['Svedeno8']=res1['df_Channel_funnel8'].astype(str)+' : '+res1['df_Campaign_funnel8'].astype(str)
res1['Svedeno9']=res1['df_Channel_funnel9'].astype(str)+' : '+res1['df_Campaign_funnel9'].astype(str)
res1['Svedeno10']=res1['df_Channel_funnel10'].astype(str)+' : '+res1['df_Campaign_funnel10'].astype(str)
res1['Svedeno11']=res1['df_Channel_funnel11'].astype(str)+' : '+res1['df_Campaign_funnel11'].astype(str)
res1['Svedeno12']=res1['df_Channel_funnel12'].astype(str)+' : '+res1['df_Campaign_funnel12'].astype(str)
res1['Svedeno13']=res1['df_Channel_funnel13'].astype(str)+' : '+res1['df_Campaign_funnel13'].astype(str)
res1['Svedeno14']=res1['df_Channel_funnel14'].astype(str)+' : '+res1['df_Campaign_funnel14'].astype(str)
res1['Svedeno15']=res1['df_Channel_funnel15'].astype(str)+' : '+res1['df_Campaign_funnel15'].astype(str)
res1['Svedeno16']=res1['df_Channel_funnel16'].astype(str)+' : '+res1['df_Campaign_funnel16'].astype(str)
res1['Svedeno17']=res1['df_Channel_funnel17'].astype(str)+' : '+res1['df_Campaign_funnel17'].astype(str)
res1['Svedeno18']=res1['df_Channel_funnel18'].astype(str)+' : '+res1['df_Campaign_funnel18'].astype(str)
res1['Svedeno19']=res1['df_Channel_funnel19'].astype(str)+' : '+res1['df_Campaign_funnel19'].astype(str)
res1['Svedeno20']=res1['df_Channel_funnel20'].astype(str)+' : '+res1['df_Campaign_funnel20'].astype(str)
res1['Svedeno21']=res1['df_Channel_funnel21'].astype(str)+' : '+res1['df_Campaign_funnel21'].astype(str)
res1['Svedeno22']=res1['df_Channel_funnel22'].astype(str)+' : '+res1['df_Campaign_funnel22'].astype(str)
res1['Svedeno23']=res1['df_Channel_funnel23'].astype(str)+' : '+res1['df_Campaign_funnel23'].astype(str)
res1['Svedeno24']=res1['df_Channel_funnel24'].astype(str)+' : '+res1['df_Campaign_funnel24'].astype(str)
res1['Svedeno25']=res1['df_Channel_funnel25'].astype(str)+' : '+res1['df_Campaign_funnel25'].astype(str)
res1['Svedeno26']=res1['df_Channel_funnel26'].astype(str)+' : '+res1['df_Campaign_funnel26'].astype(str)
res1['Svedeno27']=res1['df_Channel_funnel27'].astype(str)+' : '+res1['df_Campaign_funnel27'].astype(str)
res1['Svedeno28']=res1['df_Channel_funnel28'].astype(str)+' : '+res1['df_Campaign_funnel28'].astype(str)
res1['Svedeno29']=res1['df_Channel_funnel29'].astype(str)+' : '+res1['df_Campaign_funnel29'].astype(str)
res1['Svedeno30']=res1['df_Channel_funnel30'].astype(str)+' : '+res1['df_Campaign_funnel30'].astype(str)
res1['Svedeno31']=res1['df_Channel_funnel31'].astype(str)+' : '+res1['df_Campaign_funnel31'].astype(str)
res1['Svedeno32']=res1['df_Channel_funnel32'].astype(str)+' : '+res1['df_Campaign_funnel32'].astype(str)
res1['Svedeno33']=res1['df_Channel_funnel33'].astype(str)+' : '+res1['df_Campaign_funnel33'].astype(str)
res1['Svedeno34']=res1['df_Channel_funnel34'].astype(str)+' : '+res1['df_Campaign_funnel34'].astype(str)
res1['Svedeno35']=res1['df_Channel_funnel35'].astype(str)+' : '+res1['df_Campaign_funnel35'].astype(str)
res1['Svedeno36']=res1['df_Channel_funnel36'].astype(str)+' : '+res1['df_Campaign_funnel36'].astype(str)
res1['Svedeno37']=res1['df_Channel_funnel37'].astype(str)+' : '+res1['df_Campaign_funnel37'].astype(str)
res1['Svedeno38']=res1['df_Channel_funnel38'].astype(str)+' : '+res1['df_Campaign_funnel38'].astype(str)
res1['Svedeno39']=res1['df_Channel_funnel39'].astype(str)+' : '+res1['df_Campaign_funnel39'].astype(str)
res1['Svedeno40']=res1['df_Channel_funnel40'].astype(str)+' : '+res1['df_Campaign_funnel40'].astype(str)
res1['Svedeno41']=res1['df_Channel_funnel41'].astype(str)+' : '+res1['df_Campaign_funnel41'].astype(str)
res1['Svedeno42']=res1['df_Channel_funnel42'].astype(str)+' : '+res1['df_Campaign_funnel42'].astype(str)

In [54]:
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,df_Campaign_funnel0,df_Campaign_funnel1,df_Campaign_funnel2,df_Campaign_funnel3,...,Svedeno33,Svedeno34,Svedeno35,Svedeno36,Svedeno37,Svedeno38,Svedeno39,Svedeno40,Svedeno41,Svedeno42
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,(not set),None,None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
1,0000458812883559498,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,(not set),None,None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
3,0001436786657417059,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,(not set),None,None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,(not set),None,None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,(not set),(not set),None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62983,9999520059803839410,organic,google,NaN,NaN,1.0,(not set),None,None,None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None


In [55]:
res1.to_csv('results_campain.csv')

In [56]:
del res1['df_Campaign_funnel0']
del res1['df_Campaign_funnel1']
del res1['df_Campaign_funnel2']
del res1['df_Campaign_funnel3']
del res1['df_Campaign_funnel4']
del res1['df_Campaign_funnel5']
del res1['df_Campaign_funnel6']
del res1['df_Campaign_funnel7']
del res1['df_Campaign_funnel8']
del res1['df_Campaign_funnel9']
del res1['df_Campaign_funnel10']
del res1['df_Campaign_funnel11']
del res1['df_Campaign_funnel12']
del res1['df_Campaign_funnel13']
del res1['df_Campaign_funnel14']
del res1['df_Campaign_funnel15']
del res1['df_Campaign_funnel16']
del res1['df_Campaign_funnel17']
del res1['df_Campaign_funnel18']
del res1['df_Campaign_funnel19']
del res1['df_Campaign_funnel20']
del res1['df_Campaign_funnel21']
del res1['df_Campaign_funnel22']
del res1['df_Campaign_funnel23']
del res1['df_Campaign_funnel24']
del res1['df_Campaign_funnel25']
del res1['df_Campaign_funnel26']
del res1['df_Campaign_funnel27']
del res1['df_Campaign_funnel28']
del res1['df_Campaign_funnel29']
del res1['df_Campaign_funnel30']
del res1['df_Campaign_funnel31']
del res1['df_Campaign_funnel32']
del res1['df_Campaign_funnel33']
del res1['df_Campaign_funnel34']
del res1['df_Campaign_funnel35']
del res1['df_Campaign_funnel36']
del res1['df_Campaign_funnel37']
del res1['df_Campaign_funnel38']
del res1['df_Campaign_funnel39']
del res1['df_Campaign_funnel40']
del res1['df_Campaign_funnel41']
del res1['df_Campaign_funnel42']
del res1['df_Channel_funnel0']
del res1['df_Channel_funnel1']
del res1['df_Channel_funnel2']
del res1['df_Channel_funnel3']
del res1['df_Channel_funnel4']
del res1['df_Channel_funnel5']
del res1['df_Channel_funnel6']
del res1['df_Channel_funnel7']
del res1['df_Channel_funnel8']
del res1['df_Channel_funnel9']
del res1['df_Channel_funnel10']
del res1['df_Channel_funnel11']
del res1['df_Channel_funnel12']
del res1['df_Channel_funnel13']
del res1['df_Channel_funnel14']
del res1['df_Channel_funnel15']
del res1['df_Channel_funnel16']
del res1['df_Channel_funnel17']
del res1['df_Channel_funnel18']
del res1['df_Channel_funnel19']
del res1['df_Channel_funnel20']
del res1['df_Channel_funnel21']
del res1['df_Channel_funnel22']
del res1['df_Channel_funnel23']
del res1['df_Channel_funnel24']
del res1['df_Channel_funnel25']
del res1['df_Channel_funnel26']
del res1['df_Channel_funnel27']
del res1['df_Channel_funnel28']
del res1['df_Channel_funnel29']
del res1['df_Channel_funnel30']
del res1['df_Channel_funnel31']
del res1['df_Channel_funnel32']
del res1['df_Channel_funnel33']
del res1['df_Channel_funnel34']
del res1['df_Channel_funnel35']
del res1['df_Channel_funnel36']
del res1['df_Channel_funnel37']
del res1['df_Channel_funnel38']
del res1['df_Channel_funnel39']
del res1['df_Channel_funnel40']
del res1['df_Channel_funnel41']
del res1['df_Channel_funnel42']

In [57]:
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,Svedeno0,Svedeno1,Svedeno2,Svedeno3,...,Svedeno33,Svedeno34,Svedeno35,Svedeno36,Svedeno37,Svedeno38,Svedeno39,Svedeno40,Svedeno41,Svedeno42
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,Social : (not set),None : None,None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
1,0000458812883559498,organic,google,NaN,NaN,1.0,Organic Search : (not set),None : None,None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,Referral : (not set),None : None,None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
3,0001436786657417059,organic,google,NaN,NaN,1.0,Organic Search : (not set),None : None,None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,Direct : (not set),None : None,None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,Organic Search : (not set),None : None,None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,Referral : (not set),None : None,None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,Referral : (not set),Organic Search : (not set),None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62983,9999520059803839410,organic,google,NaN,NaN,1.0,Organic Search : (not set),None : None,None : None,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None


In [58]:
import numpy as np

In [59]:
res1["Svedeno2"].replace({"None : None": np.nan}, inplace=True)

In [60]:
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,Svedeno0,Svedeno1,Svedeno2,Svedeno3,...,Svedeno33,Svedeno34,Svedeno35,Svedeno36,Svedeno37,Svedeno38,Svedeno39,Svedeno40,Svedeno41,Svedeno42
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,Social : (not set),None : None,NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
1,0000458812883559498,organic,google,NaN,NaN,1.0,Organic Search : (not set),None : None,NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,Referral : (not set),None : None,NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
3,0001436786657417059,organic,google,NaN,NaN,1.0,Organic Search : (not set),None : None,NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,Direct : (not set),None : None,NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,Organic Search : (not set),None : None,NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,Referral : (not set),None : None,NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,Referral : (not set),Organic Search : (not set),NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None
62983,9999520059803839410,organic,google,NaN,NaN,1.0,Organic Search : (not set),None : None,NaN,None : None,...,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None,None : None


In [61]:
res1["Svedeno0"].replace({"None : None": np.nan}, inplace=True)

In [62]:
res1["Svedeno1"].replace({"None : None": np.nan}, inplace=True)

In [63]:
res1["Svedeno3"].replace({"None : None": np.nan}, inplace=True)

In [64]:
res1["Svedeno4"].replace({"None : None": np.nan}, inplace=True)

In [65]:
res1["Svedeno5"].replace({"None : None": np.nan}, inplace=True)

In [66]:
res1["Svedeno6"].replace({"None : None": np.nan}, inplace=True)

In [67]:
res1["Svedeno7"].replace({"None : None": np.nan}, inplace=True)

In [68]:
res1["Svedeno8"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno9"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno10"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno11"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno12"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno13"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno14"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno15"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno16"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno17"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno18"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno19"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno20"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno21"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno22"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno23"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno24"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno25"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno26"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno27"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno28"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno29"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno30"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno31"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno32"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno33"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno34"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno35"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno36"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno37"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno38"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno39"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno40"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno41"].replace({"None : None": np.nan}, inplace=True)
res1["Svedeno42"].replace({"None : None": np.nan}, inplace=True)

In [69]:
source_col_loc = res1.columns.get_loc('Svedeno0')
source_col_loc

6

In [70]:
source_col_loc1 = res1.columns.get_loc('Svedeno42')
source_col_loc1

48

In [71]:
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,Svedeno0,Svedeno1,Svedeno2,Svedeno3,...,Svedeno33,Svedeno34,Svedeno35,Svedeno36,Svedeno37,Svedeno38,Svedeno39,Svedeno40,Svedeno41,Svedeno42
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,Social : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000458812883559498,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,Referral : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001436786657417059,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,Direct : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,Referral : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,Referral : (not set),Organic Search : (not set),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62983,9999520059803839410,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,Svedeno0,Svedeno1,Svedeno2,Svedeno3,...,Svedeno33,Svedeno34,Svedeno35,Svedeno36,Svedeno37,Svedeno38,Svedeno39,Svedeno40,Svedeno41,Svedeno42
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,Social : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000458812883559498,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,Referral : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001436786657417059,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,Direct : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,Referral : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,Referral : (not set),Organic Search : (not set),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62983,9999520059803839410,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#res2 = res1.stack().groupby(level=0).agg('>'.join)
#res2['path'] = res2['Svedeno1'].str.cat(res2['Svedeno2'], res2['Svedeno3'],sep=" ")
#res2
#res2['path'] = res2[['Svedeno1', 'Svedeno2', 'Svedeno3', 'Svedeno4',
#                    'Svedeno5', 'Svedeno6', 'Svedeno7', 'Svedeno8',
#                    'Svedeno9', 'Svedeno10', 'Svedeno11', 'Svedeno12',
#                    'Svedeno13', 'Svedeno14', 'Svedeno15', 'Svedeno16',
#                    'Svedeno17', 'Svedeno18', 'Svedeno19', 'Svedeno20',
#                    'Svedeno21', 'Svedeno22', 'Svedeno23', 'Svedeno24',
 #                   'Svedeno25', 'Svedeno26', 'Svedeno27', 'Svedeno28',
 #                   'Svedeno29', 'Svedeno30', 'Svedeno31', 'Svedeno32',
 #                   'Svedeno33', 'Svedeno34', 'Svedeno35', 'Svedeno36',
 #                   'Svedeno37', 'Svedeno38', 'Svedeno39', 'Svedeno40',
  #                  'Svedeno41', 'Svedeno42']].dropna().astype(str).agg(' > '.join, axis=1)
#res2




res1['path'] = res1[res1.columns[6:]].apply(
    lambda x: ' > '.join(x.dropna(how='all').astype(str)),
    axis=1
)

In [76]:
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,Svedeno0,Svedeno1,Svedeno2,Svedeno3,...,Svedeno34,Svedeno35,Svedeno36,Svedeno37,Svedeno38,Svedeno39,Svedeno40,Svedeno41,Svedeno42,path
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,Social : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Social : (not set)
1,0000458812883559498,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Search : (not set)
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,Referral : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Referral : (not set)
3,0001436786657417059,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Search : (not set)
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,Direct : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Direct : (not set)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Search : (not set)
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,Referral : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Referral : (not set)
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,Referral : (not set),Organic Search : (not set),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Referral : (not set) > Organic Search : (not set)
62983,9999520059803839410,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Search : (not set)


In [77]:
res1[res1['Google_client_Id'].isin(['0000197671390269035'])]

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,Svedeno0,Svedeno1,Svedeno2,Svedeno3,...,Svedeno34,Svedeno35,Svedeno36,Svedeno37,Svedeno38,Svedeno39,Svedeno40,Svedeno41,Svedeno42,path
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,Social : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Social : (not set)


In [78]:
res1["path"].replace({"": np.nan}, inplace=True)
res1

,Google_client_Id,Medium_funnel,Source_funnel,Transactions,Revenue,New_Transactions,Svedeno0,Svedeno1,Svedeno2,Svedeno3,...,Svedeno34,Svedeno35,Svedeno36,Svedeno37,Svedeno38,Svedeno39,Svedeno40,Svedeno41,Svedeno42,path
0,0000197671390269035,referral,m.facebook.com,NaN,NaN,1.0,Social : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Social : (not set)
1,0000458812883559498,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Search : (not set)
2,0000572434142265465,(none),(direct),NaN,NaN,1.0,Referral : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Referral : (not set)
3,0001436786657417059,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Search : (not set)
4,0001611849527956932,(none),(direct),NaN,NaN,1.0,Direct : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Direct : (not set)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62980,9999350616287088509,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Search : (not set)
62981,9999355589118867468,referral,analytics.google.com,NaN,NaN,1.0,Referral : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Referral : (not set)
62982,9999355589118867468,referral > organic,analytics.google.com > google,NaN,NaN,1.0,Referral : (not set),Organic Search : (not set),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Referral : (not set) > Organic Search : (not set)
62983,9999520059803839410,organic,google,NaN,NaN,1.0,Organic Search : (not set),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Search : (not set)


In [79]:
col_Transactions = res1.groupby('path').sum().reset_index()
col_Transactions.sort_values(by='Transactions', ascending=False)

,path,Transactions,Revenue,New_Transactions
32,Direct : (not set),314.0,31938.92,14828.0
332,Organic Search : (not set),138.0,10302.27,24937.0
616,Referral : (not set),135.0,12566.75,4741.0
45,Direct : (not set) > Direct : (not set),101.0,15094.07,2253.0
629,Referral : (not set) > Direct : (not set),67.0,5663.87,750.0
...,...,...,...,...
265,Direct : (not set) > Paid Search : AW - Dynami...,0.0,0.00,1.0
268,Direct : (not set) > Referral : (not set) > Af...,0.0,0.00,2.0
270,Direct : (not set) > Referral : (not set) > Di...,0.0,0.00,13.0
271,Direct : (not set) > Referral : (not set) > Di...,0.0,0.00,6.0


In [143]:
#col_Transactions.to_csv('col_Transactions.csv')

In [80]:
path = col_Transactions

In [81]:
path = path.rename(columns={'Transactions': 'total_conversions', 'Revenue': 'total_conversion_value', 'New_Transactions': 'total_null'})

In [82]:
path.sort_values(by='total_conversions', ascending=False)

,path,total_conversions,total_conversion_value,total_null
32,Direct : (not set),314.0,31938.92,14828.0
332,Organic Search : (not set),138.0,10302.27,24937.0
616,Referral : (not set),135.0,12566.75,4741.0
45,Direct : (not set) > Direct : (not set),101.0,15094.07,2253.0
629,Referral : (not set) > Direct : (not set),67.0,5663.87,750.0
...,...,...,...,...
265,Direct : (not set) > Paid Search : AW - Dynami...,0.0,0.00,1.0
268,Direct : (not set) > Referral : (not set) > Af...,0.0,0.00,2.0
270,Direct : (not set) > Referral : (not set) > Di...,0.0,0.00,13.0
271,Direct : (not set) > Referral : (not set) > Di...,0.0,0.00,6.0


In [83]:
path.set_index('path', inplace=True)

In [84]:
path

,total_conversions,total_conversion_value,total_null
path,,,
Affiliates : Data Share Promo,0.0,0.0,969.0
Affiliates : Data Share Promo > Direct : (not set),0.0,0.0,109.0
Affiliates : Data Share Promo > Direct : (not set) > Direct : (not set),0.0,0.0,26.0
Affiliates : Data Share Promo > Direct : (not set) > Direct : (not set) > Direct : (not set),0.0,0.0,8.0
Affiliates : Data Share Promo > Direct : (not set) > Direct : (not set) > Direct : (not set) > Direct : (not set),0.0,0.0,2.0
...,...,...,...
Social : (not set) > Referral : (not set) > Social : (not set),0.0,0.0,7.0
Social : (not set) > Referral : (not set) > Social : (not set) > Social : (not set),0.0,0.0,2.0
Social : (not set) > Social : (not set),0.0,0.0,22.0


In [85]:
path[path['total_conversions'].isin(['NaN'])]

,total_conversions,total_conversion_value,total_null
path,,,


In [86]:
path.to_csv('results_campain1.csv')

In [87]:
import mta
from mta import MTA

In [88]:
mta = MTA(data='C:\\Users\\chafi\\results_campain1.csv', allow_loops = False)

running remove_loops.. elapsed time: 0.106 sec


In [96]:
mta.first_touch()
mta.last_touch()
mta.markov()
mta.shapley()
mta.shao()
MTADF = pd.DataFrame.from_dict(mta.attribution).to_csv('mta_campain_results.csv')

running first_touch.. elapsed time: 0.025 sec
running last_touch.. elapsed time: 0.025 sec
running markov.. elapsed time: 0.074 sec
running shapley.. elapsed time: 0.043 sec
running shao.. elapsed time: 0.023 sec


In [97]:
mta.show()

                                                  last_touch    linear  \
(Other) : (not set)                                 0.000000       NaN   
Affiliates : Data Share Promo                       0.000000       NaN   
Direct : (not set)                                  0.665461  0.579641   
Display : (not set)                                 0.003617  0.004370   
Organic Search : (not set)                          0.138336  0.175934   
Paid Search : (not set)                             0.000000       NaN   
Paid Search : AW - Accessories                      0.018987  0.020268   
Paid Search : AW - Apparel                          0.000000       NaN   
Paid Search : AW - Dynamic Search Ads Whole Site    0.003617  0.005651   
Paid Search : AW - Electronics                      0.000000       NaN   
Referral : (not set)                                0.152803  0.195148   
Social : (not set)                                  0.017179  0.018987   

                                     